In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.pipeline import Pipeline
from score_auto_gbm.FeatureTransformer import FeatureTransformer

In [2]:
in_data = pd.read_json("train_data.json", orient = "records")
X = in_data.drop("risk", 1)
y = np.array(in_data["risk"])

In [3]:
preprocess = FeatureTransformer()

In [4]:
gbm = GradientBoostingRegressor(learning_rate = 0.1, 
                               random_state = 1234)

In [6]:
pipe = Pipeline([("preprocess", preprocess), ("gbm", gbm)])

In [7]:
gbm_cv = GridSearchCV(pipe,
                     dict(gbm__n_estimators = [50, 100, 150, 200],
                         gbm__max_depth = [5, 6, 7, 8, 9, 10]),
                     cv = 5,
                     scoring = make_scorer(mean_squared_error),
                     verbose = 100)
gbm_cv.fit(X, y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] gbm__max_depth=5, gbm__n_estimators=50 ..........................
[CV]  gbm__max_depth=5, gbm__n_estimators=50, score=1.390660, total=   0.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[CV] gbm__max_depth=5, gbm__n_estimators=50 ..........................
[CV]  gbm__max_depth=5, gbm__n_estimators=50, score=0.884076, total=   0.1s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[CV] gbm__max_depth=5, gbm__n_estimators=50 ..........................
[CV]  gbm__max_depth=5, gbm__n_estimators=50, score=0.498074, total=   0.1s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[CV] gbm__max_depth=5, gbm__n_estimators=50 ..........................
[CV]  gbm__max_depth=5, gbm__n_estimators=50, score=0.800251, total=   0.1s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[CV] gbm__max_depth=5, gbm__n_

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('preprocess', FeatureTransformer(transforms=[('impute', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('scale', StandardScaler(copy=True, with_mean=True, with_std=True))])), ('gbm', GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
   ...       presort='auto', random_state=1234, subsample=1.0, verbose=0,
             warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'gbm__max_depth': [5, 6, 7, 8, 9, 10], 'gbm__n_estimators': [50, 100, 150, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(mean_squared_error), verbose=100)

In [8]:
import cPickle
with open("gbmFit.pkl", "wb") as pickle_file:
    cPickle.dump(gbm_cv.best_estimator_, pickle_file)